In [2]:
import src.main as lc
import torch, os

# Filepaths
HHD = "/volumes/Ultra Touch"
MLP_LOC = HHD + "/MLP/LC_test"
COMPRESSED_SAVELOC = HHD + "/MLP/Compressed"
DECOMPRESSED_SAVELOC = HHD + "/MLP/Decompressed_test"

# Base dictionary (for decompressor to understand structure of the model)
BASE_DICT = torch.load(MLP_LOC + "/frame0.tar", map_location = torch.device('cpu'))["network_fn_state_dict"]

# 1. Compression of MLP model set

In [3]:
lc.compress_set(MLP_LOC, COMPRESSED_SAVELOC)

Loading: /volumes/Ultra Touch/MLP/LC_test/frame0.tar
Delta Compression on: frame1.tar
Loading: /volumes/Ultra Touch/MLP/LC_test/frame1.tar
Delta Compression on: frame2.tar
Loading: /volumes/Ultra Touch/MLP/LC_test/frame2.tar
Delta Compression on: frame3.tar
Loading: /volumes/Ultra Touch/MLP/LC_test/frame3.tar
Delta Compression on: frame4.tar
Loading: /volumes/Ultra Touch/MLP/LC_test/frame4.tar
Saving Compressed Format: frame0.tar
Saving Compressed Format: frame1.tar
Saving Compressed Format: frame2.tar
Saving Compressed Format: frame3.tar
Saving Compressed Format: frame4.tar


# 2. Decompression from Compressed Set

In [4]:
lc.load_compressed_set(COMPRESSED_SAVELOC, DECOMPRESSED_SAVELOC, BASE_DICT)

Decompressing for: compressed_frame0.pt
Decompressing for: compressed_frame1.pt
Decompressing for: compressed_frame2.pt
Decompressing for: compressed_frame3.pt
Decompressing for: compressed_frame4.pt
Saving Decompressed Model at: decompressed_frame0.pt
Saving Decompressed Model at: decompressed_frame1.pt
Saving Decompressed Model at: decompressed_frame2.pt
Saving Decompressed Model at: decompressed_frame3.pt
Saving Decompressed Model at: decompressed_frame4.pt


# 3. Testing Random Weights in decompressed MLP

In [5]:
frame_no = 3
original = torch.load(MLP_LOC + "/frame{}.tar".format(frame_no), map_location = torch.device('cpu'))["network_fn_state_dict"]
compressed = lc.read_decompressed_state_dict(DECOMPRESSED_SAVELOC + "/decompressed_frame{}.pt".format(frame_no))

In [6]:
original['pts_linears.1.weight'][2][10:20]

tensor([-0.1628, -0.0399,  0.0876, -0.0959,  0.2464, -0.0933, -0.0802,  0.0091,
         0.1737, -0.0667])

In [7]:
compressed['pts_linears.1.weight'][2][10:20]

tensor([-0.1436, -0.0426,  0.0437, -0.0939,  0.2938, -0.0950, -0.0698,  0.0255,
         0.1078, -0.0235])

In [8]:
original['pts_linears.2.weight'][4][10:20]

tensor([-0.0092,  0.0341, -0.3084, -0.0638,  0.0502,  0.1596, -0.0123,  0.0431,
        -0.0161,  0.1263])

In [9]:
compressed['pts_linears.2.weight'][4][10:20]

tensor([-0.0399,  0.0949, -0.3225, -0.1085,  0.0502,  0.1478, -0.0327,  0.0424,
        -0.0353,  0.1362])

In [10]:
frame_no = 1
original = torch.load(MLP_LOC + "/frame{}.tar".format(frame_no), map_location = torch.device('cpu'))["network_fn_state_dict"]
compressed = lc.read_decompressed_state_dict(DECOMPRESSED_SAVELOC + "/decompressed_frame{}.pt".format(frame_no))

In [11]:
original['pts_linears.1.weight'][4][5:15]

tensor([-0.2480, -0.2322,  0.0630, -0.1238, -0.0873,  0.1134,  0.1539,  0.0085,
         0.0692, -0.0862])

In [12]:
compressed['pts_linears.1.weight'][4][5:15]

tensor([-0.1702, -0.2176,  0.0619, -0.1270, -0.0907,  0.1337,  0.1742,  0.0024,
         0.0702, -0.0826])

# 4. Compression Ratio

In [13]:
uncompressed_size = sum(os.path.getsize(MLP_LOC + "/" + f) for f in os.listdir(MLP_LOC))
compressed_size = sum(os.path.getsize(COMPRESSED_SAVELOC + "/" + f) for f in os.listdir(COMPRESSED_SAVELOC))
print("Compression Ratio:")
print(round((uncompressed_size - compressed_size) / uncompressed_size, 5))

Compression Ratio:
0.94242


# 5. Save the torch dicts

In [16]:
for i in range(0, 5):
    compressed = lc.read_decompressed_state_dict(DECOMPRESSED_SAVELOC + "/decompressed_frame{}.pt".format(i))
    torch.save(compressed, HHD + "/MLP/test/frame_{}.pt".format(i))

In [18]:
torch.load(HHD + "/MLP/test/frame_{}.pt".format(1))

OrderedDict([('pts_linears.0.weight',
              tensor([[-0.0022, -0.0335,  0.0301,  ...,  0.0461, -0.0409,  0.0100],
                      [ 0.2422, -0.0641,  0.0302,  ...,  0.0691,  0.0836, -0.0046],
                      [-0.0082,  0.0983, -0.2047,  ..., -0.0366, -0.0359,  0.0588],
                      ...,
                      [-0.1444, -0.0826,  0.1752,  ..., -0.0106, -0.0008, -0.0450],
                      [ 0.0584, -0.0534,  0.1914,  ...,  0.0087,  0.0043, -0.0202],
                      [-0.0315,  0.1031,  0.2622,  ...,  0.0276, -0.0058, -0.0316]])),
             ('pts_linears.0.bias',
              tensor([-0.1914, -0.0888, -0.0006, -0.0771,  0.0545, -0.0870, -0.1396, -0.0638,
                      -0.0271, -0.0009, -0.0166,  0.0016, -0.0371, -0.1509, -0.0781, -0.0314,
                      -0.0961, -0.0681,  0.0088, -0.0805, -0.0695, -0.1869, -0.1795,  0.0177,
                       0.0331,  0.1317, -0.2049,  0.0218,  0.0045, -0.0240, -0.0983,  0.1132,
                